# G12 | DMYTRO VERNYUK Y MIKHAIL RONCHYK

# Practica 2 - Parte 2: Recuperación de información

In [11]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import numpy.ma as ma
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
rand = 911 # Numero para los parametros aleatorios

# Apartado a)

In [3]:
train_data = fetch_20newsgroups(subset='train', shuffle=True, random_state=rand)
test_data = fetch_20newsgroups(subset='test')

In [4]:
with open('words.txt') as f:
    dictionary = f.read().splitlines()

# Bolsa de palabras
vectorizer = CountVectorizer(vocabulary=dictionary, stop_words='english', 
                             binary=False, ngram_range=(1,1))

train_vector_data = vectorizer.fit_transform(train_data.data)
test_vector_data = vectorizer.transform(test_data.data)

In [5]:
#Ordena de mayor a menor
def ordenar(index,val,target,array):
    if len(array) == index:
        return array
    
    elif array[index,0] == 0:
        array[index,0] = val
        array[index,1] = target
        return array
    
    elif val > array[index,0]:
        aux_val = array[index,0]
        aux_tar = array[index,1]
        array[index,0] = val
        array[index,1] = target
        return ordenar(index+1,aux_val,aux_tar,array)
    
    else:
        return ordenar(index+1,val,target,array)

In [9]:
# Media de los 3 primeros terminos de test_data
def media(target,exh,test_vector,train_vector):
    ret = 0
    i = 0
    x = 0
    array_aux = np.full((exh,2),0.0)

    while i < 3:
        if test_data.target[x] != target:
            x += 1
        else:
            i += 1
            x += 1
            
            #Calculo de todo train_vector con el mensaje x de test_vector
            for y in range(0, len(train_data.target)): #me quiero morir
                cs = cosine_similarity(test_vector[x],train_vector[y])
                array_aux = ordenar(0, cs, train_data.target[y], array_aux)
           
            for k in range(0, exh):
                ret_aux = 0
                if array_aux[k,1] == test_data.target[x]:
                    ret_aux += 1    
            ret += (ret_aux / exh) #Porcentaje de aciertos para el mensaje x
            
            array_aux = np.full((exh,2),0.0)
    
    ret /= 3 #La media de los 3 porcentajes
    
    return ret

In [10]:
# Imprime la media de cada target con exhautividad 3; tarda en ejecutar más que un parto
for x in range(0, len(train_data.target_names)):
    print('*', train_data.target_names[x], '~>', "%.2f" % media(x, 3, 
                                                                test_vector_data, 
                                                                train_vector_data))

* alt.atheism ~> 0.11
* comp.graphics ~> 0.22
* comp.os.ms-windows.misc ~> 0.11
* comp.sys.ibm.pc.hardware ~> 0.11
* comp.sys.mac.hardware ~> 0.22
* comp.windows.x ~> 0.11
* misc.forsale ~> 0.11
* rec.autos ~> 0.11
* rec.motorcycles ~> 0.11
* rec.sport.baseball ~> 0.11
* rec.sport.hockey ~> 0.22
* sci.crypt ~> 0.00
* sci.electronics ~> 0.22
* sci.med ~> 0.33
* sci.space ~> 0.22
* soc.religion.christian ~> 0.11
* talk.politics.guns ~> 0.22
* talk.politics.mideast ~> 0.22
* talk.politics.misc ~> 0.11
* talk.religion.misc ~> 0.22


In [251]:
# Imprime la media de cada target con exhautividad 10; tarda en ejecutar más que un parto
for x in range(0, len(train_data.target_names)):
    print('*', train_data.target_names[x], '~>', "%.2f" % media(x, 10, 
                                                                test_vector_data, 
                                                                train_vector_data))

* alt.atheism ~> 0.03
* comp.graphics ~> 0.07
* comp.os.ms-windows.misc ~> 0.03
* comp.sys.ibm.pc.hardware ~> 0.03
* comp.sys.mac.hardware ~> 0.00
* comp.windows.x ~> 0.10
* misc.forsale ~> 0.03
* rec.autos ~> 0.03
* rec.motorcycles ~> 0.03
* rec.sport.baseball ~> 0.00
* rec.sport.hockey ~> 0.10
* sci.crypt ~> 0.00
* sci.electronics ~> 0.03
* sci.med ~> 0.07
* sci.space ~> 0.03
* soc.religion.christian ~> 0.03
* talk.politics.guns ~> 0.03
* talk.politics.mideast ~> 0.07
* talk.politics.misc ~> 0.03
* talk.religion.misc ~> 0.07


- # ¿Hay muchas diferencias entre los valores de precisión medios para las distintas clases del conjunto de datos? ¿A qué crees que se deben?

    Por el vocabulario especifico de cada rama. En la clase sci.med o 
    rec.sport.hockey tenemos una serie de tecnicismos propios que ayudan a 
    discriminar de una forma mucho mas efectiva.

- # Identifica la clase que haya tenido peores resultados de precisión y para alguna de sus consultas muestra alguno de los mensajes que recuperó erróneamente en las primeras X posiciones.

    sci.crypt no ha mostrado en ninguno de los dos casos una media positiva.
    No se como mostrar las recuperaciones erroneas.

- # ¿Con qué clases se ha confundido más dicha consulta?

        N/D

- # ¿A qué crees que se deben los malos resultados?

    No es la forma adecuada de recuperar información. Usar solo 
    menjes especificos no sirve ya que estos mismos pueden variar en un 
    mismo tema.

# Apartado b)

In [14]:
# TF-IDF 
tfidfer = TfidfTransformer()
train_preprocessed = tfidfer.fit_transform(train_vector_data)
test_preprocessed = tfidfer.transform(test_vector_data)

In [15]:
# Imprime la media de cada target con exhautividad 3; tarda en ejecutar más que un parto
for x in range(0, len(train_data.target_names)):
    print('*', train_data.target_names[x], '~>', "%.2f" % media(x, 3,
                                                                test_preprocessed, 
                                                                train_preprocessed))

* alt.atheism ~> 0.00
* comp.graphics ~> 0.00
* comp.os.ms-windows.misc ~> 0.11
* comp.sys.ibm.pc.hardware ~> 0.11
* comp.sys.mac.hardware ~> 0.00
* comp.windows.x ~> 0.11
* misc.forsale ~> 0.22
* rec.autos ~> 0.22
* rec.motorcycles ~> 0.00
* rec.sport.baseball ~> 0.00
* rec.sport.hockey ~> 0.33
* sci.crypt ~> 0.11
* sci.electronics ~> 0.22
* sci.med ~> 0.33
* sci.space ~> 0.22
* soc.religion.christian ~> 0.33
* talk.politics.guns ~> 0.22
* talk.politics.mideast ~> 0.22
* talk.politics.misc ~> 0.22
* talk.religion.misc ~> 0.11


In [16]:
# Imprime la media de cada target con exhautividad 10; tarda en ejecutar más que un parto
for x in range(0, len(train_data.target_names)):
    print('*', train_data.target_names[x], '~>', "%.2f" % media(x, 10,
                                                                test_preprocessed, 
                                                                train_preprocessed))

* alt.atheism ~> 0.00
* comp.graphics ~> 0.03
* comp.os.ms-windows.misc ~> 0.03
* comp.sys.ibm.pc.hardware ~> 0.03
* comp.sys.mac.hardware ~> 0.07
* comp.windows.x ~> 0.07
* misc.forsale ~> 0.07
* rec.autos ~> 0.03
* rec.motorcycles ~> 0.07
* rec.sport.baseball ~> 0.07
* rec.sport.hockey ~> 0.03
* sci.crypt ~> 0.03
* sci.electronics ~> 0.07
* sci.med ~> 0.03
* sci.space ~> 0.03
* soc.religion.christian ~> 0.07
* talk.politics.guns ~> 0.07
* talk.politics.mideast ~> 0.07
* talk.politics.misc ~> 0.07
* talk.religion.misc ~> 0.03


- # ¿Han cambiado los valores de precisión media para las clases del conjunto de datos? ¿Qué clases han mejorado? ¿Cuáles han empeorado?

    Para exhautividad 3 en general la recuperación ha empeorado y para 
    exhautividad 10 ha mejorado. 

- # Encuentra una consulta donde el uso de la ponderación TF-IDF haya sido efectivo y haya mejorado los resultados. Explica por qué ha sido efectivo.

    sci.crypt ha mejorado significativamente, teniendo por fin un porcentaje positivo. Me imagino que es asi porque para un tema como la criptografía hay mucho ruido en el entre sus textos proveniente de otros temas ya que es naturaleza de este estudio.